In [1]:
import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.graphs.networkx_graph import KnowledgeTriple

# prompt template for knowledge triple extraction

_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE = (
    "You are a networked intelligence helping a human track knowledge triples"
    " about all relevant people, things, concetps, etc. and integrating"
    " them with your knowledge stored within your weights"
    " as well as that stored in a knowledge graph"
    " Extract all of the knowledge triples from the text"
    " A knowledge triple is a clause that contains a subject, a predicate, "
    " and an object. The subject is the entity being described,"
    " the predicate is the property of the subject that is being"
    " described, and the object is the value of the property.\n\n"
    "EXAMPLE\n"
    "It is a state in the US. It's also the number 1 producer of gold in the US.\n\n"
    f"Output: (Nevada, is a, state){KnowledgeTriple}(Nevada, is in, US)"
    f"{KnowledgeTriple}(Nevada, is the number 1 producer of, gold)\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "I'm going to the store.\n\n"
    "Output: NONE\n"
    "EXAMPLE\n"
    "Oh huh, I know Descartes likes to drive antique scooters and play the mandolin.\n"
    f"Output: (Descartes, likes to drive, antique scooters){KnowledgeTriple}(Descartes, plays, mandolin)\n"
    "END OF EXAMPLES\n"
    "EXAMPLE\n"
    "{text}"
    "Output:"
)

KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT = PromptTemplate(
    input_variables=["text"],
    template=_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE
)

# instantiate the OpenAI model
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0, openai_api_key=OPENAI_API_KEY)

# create an LLMChain using the knowledge triple extraction prompt
chain = LLMChain(llm=llm, prompt=KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT)

# run the chain with the specified text
text = "The city of Paris is the capital and most populous city of France. The Eiffel Tower is a famous landmark in Paris."
triples = chain.run(text)

print("===")
print(triples)

C:\Users\woojin\AppData\Local\Temp\ipykernel_19396\399117189.py:41: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0, openai_api_key=OPENAI_API_KEY)
C:\Users\woojin\AppData\Local\Temp\ipykernel_19396\399117189.py:44: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT)
C:\Users\woojin\AppData\Local\Temp\ipykernel_19396\399117189.py:48: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 

===
(Paris, is the capital of, France)<class 'langchain_community.graphs.networkx_graph.KnowledgeTriple'>(Paris, is the most populous city of, France)<class 'langchain_community.graphs.networkx_graph.KnowledgeTriple'>(Eiffel Tower, is a famous landmark in, Paris)


In [3]:
triples

"(Paris, is the capital of, France)<class 'langchain_community.graphs.networkx_graph.KnowledgeTriple'>(Paris, is the most populous city of, France)<class 'langchain_community.graphs.networkx_graph.KnowledgeTriple'>(Eiffel Tower, is a famous landmark in, Paris)"

In [4]:
# clean text formatted to KR_TRIPLE_DELIMITER (" | ")
import re
cleaned_triples = re.sub(r"<class.*?>", " | ", triples)

KR_TRIPLE_DELIMETER = " | "

def parse_triples(response, delimiter=KR_TRIPLE_DELIMETER):
    if not response:
        return []
    return response.split(delimiter)

triples_list = parse_triples(cleaned_triples)

# print teh extracted relation tripplets
print(triples_list)

['(Paris, is the capital of, France)', '(Paris, is the most populous city of, France)', '(Eiffel Tower, is a famous landmark in, Paris)']


# Knowlege Graph Visualization

In [7]:
triples_list

['(Paris, is the capital of, France)',
 '(Paris, is the most populous city of, France)',
 '(Eiffel Tower, is a famous landmark in, Paris)']

In [9]:
from pyvis.network import Network
import networkx as nx

# create a Network graph from the extracted relation triples
def create_graph_from_triplets(triplets):
    G = nx.DiGraph()
    for triplet in triplets:
        subject, predicate, obj = triplet.strip().split(',')
        G.add_edge(subject.strip(), obj.strip(), label=predicate.strip())
    return G

# convert the Network graph to a PyVIS network
def nx_to_pyvis(networkx_graph):
    pyvis_graph = Network(notebook=True, cdn_resources='remote')
    for node in networkx_graph.nodes():
        pyvis_graph.add_node(node)
    for edge in networkx_graph.edges(data=True):
        pyvis_graph.add_edge(edge[0], edge[1], label=edge[2]['label'])
    return pyvis_graph

triplets = [t.strip() for t in triples_list if t.strip()]
graph = create_graph_from_triplets(triplets)
pyvis_network = nx_to_pyvis(graph)

# customize the appearance of the graph
pyvis_network.toggle_hide_edges_on_drag(True)
pyvis_network.toggle_physics(False)
pyvis_network.set_edge_smooth('discrete')

# show the interactive knowledge graph visualization
pyvis_network.show("knowledge_graph.html")

knowledge_graph.html
